# Hate Speech Detector - EN - Features extraction for SVM & Dense model

Based on [this notebook](https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/classifier/final_classifier.ipynb).

In [1]:
import os
import pandas as pd
import csv
import numpy as np
import pickle
from klepto.archives import dir_archive
import sys
import nltk
import string
import re
import fasttext
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
import syllables as sylla

from tqdm.notebook import tqdm
%matplotlib inline

In [2]:
MODEL = 'dense'
dim = 10 if MODEL == 'svm' else 200

## Davidson et al. data loading

### Classes pre:
    0 - hate speech
    1 - offensive language
    2 - neither

In [3]:
if not os.path.exists('hsd/DavidsonEtAl/perfect_data.pkl'):
    tweets, labels = [], []
    with open('hsd/DavidsonEtAl/labeled_data.csv', 'r') as f:
        for d in tqdm(list(csv.reader(f))[1:]):
            tweets.append(d[6])  # tweet
            labels.append(d[5])  # class
    with open('hsd/DavidsonEtAl/perfect_data.pkl', 'w') as f:
        def chcl(c):
            return 0 if c=='2' else 1
        labels = list(map(chcl, labels))
        pickle.dump((tweets, labels), f)
else:
    with open('hsd/DavidsonEtAl/perfect_data.pkl', 'r') as f:
        tweets, labels = pickle.load(f)

### Classes post:
    0 - no hate
    1 - hate speech

In [4]:
print('Tweets: {}'.format(len(tweets)))
print('Labels: {}'.format(len(labels)))

Tweets: 24783
Labels: 24783


In [5]:
list(zip(tweets[:5], labels[:5]))

[("!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...",
  0),
 ('!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!',
  1),
 ('!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit',
  1),
 ('!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny', 1),
 ('!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;',
  1)]

## Features extraction

In [6]:
sentiment_analyzer = VS()

In [7]:
def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    #hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    #parsed_text = re.sub(hashtag_regex, '', parsed_text)
    return parsed_text

def basic_tokenize(tweet):
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet.split()

def get_pos_string(tweet):
    text = preprocess(tweet)
    tokens = basic_tokenize(preprocess(text))
    tags = nltk.pos_tag(tokens)
    tag_list = [x[1] for x in tags]
    tag_str = ' '.join(tag_list)
    
    return tag_str

def pad_words(words, length):
    if len(words) >= length:
        return words[:length]
    else:
        additional = length - len(words)
        return words + ['EMPTY']*additional

def adjust_words(words, length):
    # different from pad: output tokens may contain more than 1 words
    if len(words) >= length:
        q, r = divmod(len(words), length)
        return [' '.join(words[i * q + min(i, r):(i + 1) * q + min(i + 1, r)]) for i in xrange(length)]
    else:
        additional = length - len(words)
        return words + ['EMPTY']*additional

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

### Supervised fastText wordtokens training

In [8]:
if not os.path.exists('hsd/DavidsonEtAl/fasttext.ft'):
    with open('hsd/DavidsonEtAl/fasttext.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, preprocess(t)))

# load fasttext model or train & save if none
if os.path.exists('hsd/DavidsonEtAl/fasttext_{}.bin'.format(MODEL)):
    ft_model = fasttext.load_model('hsd/DavidsonEtAl/fasttext_{}.bin'.format(MODEL))
else:
    ft_model = fasttext.train_supervised('hsd/DavidsonEtAl/fasttext.ft',
                                         lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_model.save_model('hsd/DavidsonEtAl/fasttext_{}.bin'.format(MODEL))

### Wordtoken features

In [9]:
def get_wordtoken_fts(data, length=None):
    
    sentences_words = []
    t = tqdm(data)
    t.set_postfix_str('Wordtokens features extraction: tokenization.')
    for d in t:
        sentence = preprocess(d)
        sentences_words.append(sentence.split(' '))
    
    if length:
        sentences_words = [adjust_words(sw, length) for sw in sentences_words]
        print('Required length: {}'.format(length))
    else:
        opt_length = int(np.median([len(sw) for sw in sentences_words]))
        print('Optimal median length: {}'.format(opt_length))
        sentences_words = [pad_words(sw, opt_length) for sw in sentences_words]
    
    ft_vectors = []
    t = tqdm(sentences_words)
    t.set_postfix_str('Wordtokens features extraction: vectorization.')
    for sw in t:
        ft_vector = []
        for w in sw:
            ft_vector.extend(ft_model[w])
        ft_vectors.append(ft_vector)
    
    return ft_vectors

In [10]:
wordtoken_features = get_wordtoken_fts(tweets)


Optimal median length: 13


In [11]:
wordtoken_features[0]

[-0.005886471,
 -0.007677337,
 0.0069653876,
 0.009944246,
 -0.012278754,
 0.01757102,
 -0.011702702,
 0.005516859,
 -0.012910479,
 -0.02917462,
 0.0108544,
 -0.011088269,
 -0.013530828,
 0.006403526,
 0.0055776145,
 -0.011673365,
 -0.008483273,
 0.00069808826,
 -0.0122123575,
 0.0031394574,
 0.0032052605,
 -0.016254319,
 -0.004197111,
 0.010502181,
 -0.006176906,
 0.0051430115,
 -0.0105207395,
 0.011157191,
 -0.0028083357,
 -0.028084492,
 0.000680203,
 0.0028220254,
 0.010700878,
 0.003288889,
 0.01516075,
 -0.004095278,
 -0.006681422,
 0.016342012,
 -0.012547598,
 0.006147178,
 0.005003886,
 -0.0074912487,
 -0.013177864,
 0.0076756366,
 -0.0074083023,
 0.006369958,
 -0.006894101,
 -0.0004061483,
 0.0015611604,
 -0.01805385,
 0.019459708,
 -0.012031663,
 0.0120775895,
 -0.011576787,
 0.01019344,
 0.01189034,
 -0.0018766932,
 0.008788248,
 -0.004722748,
 -0.0130946785,
 -0.007155477,
 -0.0106154345,
 -0.014130867,
 -0.012831356,
 -0.0014254216,
 -0.014910057,
 0.01381497,
 -0.011194766

### Supervised fastText pos training

In [12]:
if not os.path.exists('hsd/DavidsonEtAl/fasttext_pos.ft'):
    with open('hsd/DavidsonEtAl/fasttext_pos.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, get_pos_string(t)))

# load fasttext pos model or train & save if none
if os.path.exists('hsd/DavidsonEtAl/fasttext_pos_{}.bin'.format(MODEL)):
    ft_pos_model = fasttext.load_model('hsd/DavidsonEtAl/fasttext_pos_{}.bin'.format(MODEL))
else:
    ft_pos_model = fasttext.train_supervised('hsd/DavidsonEtAl/fasttext_pos.ft',
                                             lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_pos_model.save_model('hsd/DavidsonEtAl/fasttext_pos_{}.bin'.format(MODEL))

### Part of speech (PoS) features

In [13]:
def get_pos_fts(data, length=None):

    #Get POS tags for tweets and save as a string
    pos_sentences = []
    t = tqdm(data)
    t.set_postfix_str('PoS features extraction: tokenization.')
    for d in t:
        pos_string = get_pos_string(d)
        pos_sentences.append(pos_string)
        
        
    pos_tags = []
    for ps in pos_sentences:
        pos_tags.append(ps.split(' '))
    
    if length:
        pos_tags = [adjust_words(pt, length) for pt in pos_tags]
        print('Required length: {}'.format(length))
    else:
        opt_length = int(np.median([len(pt) for pt in pos_tags]))
        print('Optimal median length: {}'.format(opt_length))
        pos_tags = [pad_words(pt, opt_length) for pt in pos_tags]
    
    ft_vectors = []
    t = tqdm(pos_tags)
    t.set_postfix_str('PoS features extraction: vectorization.')
    for pt in t:
        ft_vector = []
        for t in pt:
            ft_vector.extend(ft_pos_model[t])
        ft_vectors.append(ft_vector)
    
    return ft_vectors

In [14]:
pos_features = get_pos_fts(tweets)


Optimal median length: 12


In [15]:
pos_features[0]

[-0.10953482,
 0.12479462,
 -0.23941049,
 -0.033208188,
 -0.13052273,
 -0.05160633,
 0.100917816,
 -0.053835046,
 0.48181352,
 0.1884942,
 0.007934232,
 -0.00485046,
 -0.016450016,
 0.17867516,
 -0.09688176,
 0.03200966,
 0.009703484,
 0.043056283,
 0.018640947,
 0.048195943,
 -0.0070820292,
 0.028998312,
 0.052482907,
 -0.19693875,
 0.04093633,
 -0.11699067,
 0.25694934,
 -0.34839612,
 -0.09780719,
 0.09110506,
 -0.007182199,
 -0.06458191,
 -0.043675665,
 -0.13684069,
 -0.078949265,
 0.036391187,
 0.102132455,
 -0.06252775,
 0.035943888,
 0.009633291,
 0.038128685,
 0.17760171,
 0.12639698,
 0.12660192,
 0.02258985,
 0.026801046,
 0.058441807,
 -0.045209695,
 -0.23921908,
 0.16329426,
 -0.34300745,
 0.35169494,
 -0.019737594,
 0.066820376,
 -0.07374335,
 -0.09064794,
 -0.016615193,
 0.1360049,
 0.16344291,
 0.13620658,
 -0.04337546,
 0.080764,
 -0.010570508,
 0.11125387,
 -0.22090569,
 0.0608359,
 -0.0014460109,
 0.016603684,
 0.031519372,
 0.041567028,
 -0.1496009,
 -0.0053717582,
 -

### Other features

In [16]:
def get_other_fts(data):
    other_features = []
    t = tqdm(data)
    t.set_postfix_str('Other features extraction.')
    for tweet in t:
        """This function takes a string and returns a list of features.
        These include Sentiment scores, Text and Readability scores,
        as well as Twitter specific features"""
        sentiment = sentiment_analyzer.polarity_scores(tweet)

        words = preprocess(tweet) #Get text only

        syllables = sylla.estimate(words)
        num_chars = sum(len(w) for w in words)
        num_chars_total = len(tweet)
        num_terms = len(tweet.split())
        num_words = len(words.split())
        avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
        num_unique_terms = len(set(words.split()))

        ###Modified FK grade, where avg words per sentence is just num words/1
        FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59, 1)
        ##Modified FRE score, where sentence fixed to 1
        FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)), 2)

        twitter_objs = count_twitter_objs(tweet)
        retweet = 0 if "rt" in words else 1
        features = [FKRA, FRE, syllables, avg_syl, num_chars, num_chars_total, num_terms, num_words,
                    num_unique_terms, sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],
                    twitter_objs[2], twitter_objs[1],
                    twitter_objs[0], retweet]
        other_features.append(features)
    
    return np.array(other_features)

In [17]:
other_features = get_other_fts(tweets)

In [18]:
other_features[:5]

array([[ 7.4000e+00,  8.6710e+01,  2.8000e+01,  1.1200e+00,  1.2700e+02,
         1.4000e+02,  2.5000e+01,  2.5000e+01,  2.3000e+01,  0.0000e+00,
         1.2000e-01,  8.8000e-01,  4.5630e-01,  0.0000e+00,  1.0000e+00,
         0.0000e+00,  1.0000e+00],
       [ 1.7000e+00,  1.1128e+02,  1.5000e+01,  9.3750e-01,  7.7000e+01,
         8.5000e+01,  1.6000e+01,  1.6000e+01,  1.6000e+01,  2.3700e-01,
         0.0000e+00,  7.6300e-01, -6.8760e-01,  0.0000e+00,  1.0000e+00,
         0.0000e+00,  1.0000e+00],
       [ 3.4000e+00,  1.0617e+02,  1.9000e+01,  9.5000e-01,  9.3000e+01,
         1.2000e+02,  2.1000e+01,  2.0000e+01,  1.8000e+01,  5.3800e-01,
         0.0000e+00,  4.6200e-01, -9.5500e-01,  0.0000e+00,  2.0000e+00,
         0.0000e+00,  0.0000e+00],
       [-2.1000e+00,  1.2469e+02,  7.0000e+00,  8.7500e-01,  3.8000e+01,
         6.2000e+01,  9.0000e+00,  8.0000e+00,  8.0000e+00,  0.0000e+00,
         3.4400e-01,  6.5600e-01,  5.6730e-01,  0.0000e+00,  2.0000e+00,
         0.0000e+00

### All features and feature names

In [19]:
#Now join them all up
features = np.concatenate([wordtoken_features, pos_features, other_features],axis=1)

In [20]:
features.shape

(24783, 5017)

## Save features & labels

In [21]:
archive = dir_archive('hsd/DavidsonEtAl/X_y_{}'.format(MODEL), {'features': features, 'labels': labels}, serialized=True)
archive.dump()
del archive

# Hate Speech Detector - EN - Features extraction for EN test data

In [ ]:
BATCHES = 50

TOKENS_LENGTH = 13
POS_LENGTH = 12

if os.path.exists('tests/tweets_en.csv'):
    with open('tests/tweets_en.csv', 'r') as f:
        raw_tweets = [d[0] for d in tqdm(list(csv.reader(f))[1:])]
    
    q, r = divmod(len(raw_tweets), BATCHES)
    tweets_batches = [raw_tweets[i * q + min(i, r):(i + 1) * q + min(i + 1, r)] for i in xrange(BATCHES)]
    
    for batch in range(BATCHES):
        print('Batch {}/{}'.format(batch+1, BATCHES))
        tweets_batch = tweets_batches[batch]
    
        wt_features = get_wordtoken_fts(tweets_batch, length=TOKENS_LENGTH)
        p_features = get_pos_fts(tweets_batch, length=POS_LENGTH)
        o_features = get_other_fts(tweets_batch)

        all_features = np.concatenate([wt_features, p_features, o_features], axis=1)
        print('Done! Extracted dimensions: {}'.format(all_features.shape))
        
        batch_str = str(batch) if batch >= 100 else '0'+str(batch) if batch >= 10 else '00'+str(batch)
        archive = dir_archive('tests/en_{}/X_{}'.format(MODEL, batch_str), {'features': all_features}, serialized=True)
        archive.dump()
        del archive
    print('All done!')